In [1]:
!nvidia-smi

Thu Jan 23 10:05:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.02              Driver Version: 566.03         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 Ti     On  |   00000000:01:00.0  On |                  N/A |
|  0%   39C    P8              5W /  165W |     593MiB /  16380MiB |     10%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
import os
import torch
import transformers
import peft
import datasets
import evaluate
import time
assert torch.cuda.is_available(), "you need cuda for this part"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device

device(type='cuda', index=0)

In [3]:
dataset_path = "/app/datasets/oberon/docs/bb_ru"
#model_name = "Qwen/Qwen2.5-Coder-7B"
model_name = 'MTSAIR/Cotype-Nano'

EPOCHS = 7
PACKED=False

In [4]:
only_model_name = model_name.split("/")[-1].replace(':', "_")
dataset_name = dataset_path.split("/")[-1]
sft_model_path = f"/app/models/{dataset_name}_{only_model_name}"

In [5]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
peft_config = peft.LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "mlp.down_proj",
        "self_attn.k_proj",
        "self_attn.o_proj",
        "mlp.up_proj",
        "self_attn.v_proj",
        "mlp.gate_proj",
        "self_attn.q_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=peft.TaskType.CAUSAL_LM
)


In [6]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [7]:
def load_texts():
    file_names = [] 
    for subdir, dirs, files in os.walk(dataset_path):
        if "/Rsrc/" in subdir:
            continue
        for file in files:
            file_names.append(os.path.join(subdir, file))
    texts = []
    for f in file_names:
        with open(f, 'r', encoding='utf-8') as file:
            texts.append(file.read())
    return texts
texts = load_texts()



In [10]:
text_dataset = datasets.Dataset.from_dict({"texts":texts})

In [11]:

text_dataset.push_to_hub('hodza/Informatika21CP')

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.11s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/hodza/Informatika21CP/commit/d8a9343597f441c4500b1e87a9513422a20b973e', commit_message='Upload dataset', commit_description='', oid='d8a9343597f441c4500b1e87a9513422a20b973e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/hodza/Informatika21CP', endpoint='https://huggingface.co', repo_type='dataset', repo_id='hodza/Informatika21CP'), pr_revision=None, pr_num=None)

In [9]:
block_size = 256  # Максимальная длина последовательности
stride = 32      # Шаг перекрытия    
def tokenize_with_stride(texts):
    tokenized_data = tokenizer(
        texts,
        max_length=block_size,
        truncation=True,
        stride=stride,
        return_overflowing_tokens=True,
        padding="max_length",  # Дополняем до block_size
        return_tensors="pt",   # Возвращаем тензоры PyTorch
    )
    tokenized_data["labels"] = tokenized_data["input_ids"]
    return datasets.Dataset.from_dict(tokenized_data).train_test_split(test_size=0.1)
    
lm_datasets = tokenize_with_stride(texts)
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'overflow_to_sample_mapping', 'labels'],
        num_rows: 12160
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'overflow_to_sample_mapping', 'labels'],
        num_rows: 1352
    })
})

In [10]:

if False:# probaby need to remove as obsolete
    print("USING PACKED DATASET")
    def tokenize_function(examples):
        return tokenizer(examples['text'])
    
    tokenized_dataset = dataset.map(tokenize_function)
    
    def group_texts(examples):
        # Concatenate all texts.
        concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
        total_length = len(concatenated_examples[list(examples.keys())[0]])
        # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
            # customize this part to your needs.
        total_length = (total_length // block_size) * block_size
        # Split by chunks of max_len.
        result = {
            k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
            for k, t in concatenated_examples.items()
        }
        result["labels"] = result["input_ids"].copy()
        return result
    
    lm_datasets = tokenized_dataset.map(
        group_texts,
        batched=True,
        batch_size=1000,
        num_proc=4,
    )

    

In [11]:
print(tokenizer.decode(lm_datasets["train"][99]["input_ids"]))

очую сессию (всего около 10 часов, включая либеральные перерывы на чай, ужин, чтение новостей и отслеживание текущих результатов матчей проходившего тогда теннисного турнира WTA) и сразу корректно заработала, не потребовав традиционной отладки. 
Единственная ошибка (случай, когда документ кончается до точки, закрывающей модуль) была найдена просто на свежую голову (на следующее утро) верификацией программного текста, то есть, опять же, без отладки.
Разумеется, окончательный результат подвергся довольно обширному тестированию, но  это важно подчеркнуть  роль традиционной отладки как способа поиска и устранения ошибок была равна в точности нулю.
Ф.В.Ткачев (info21)
*)
		(* готовимся войти в цикл: *)
		pos := 0; (* пози


In [12]:
model = transformers.AutoModelForCausalLM.from_pretrained(model_name, device_map=device,quantization_config=bnb_config,)
model._hf_peft_config_loaded = True  # silence a warning from HF trainer

In [13]:
model = peft.get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 18,464,768 || all params: 1,562,179,072 || trainable%: 1.1820


In [14]:
import logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(message)s')

In [15]:
metric = evaluate.load("bleu")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

2025-01-23 10:05:47,827 Starting new HTTPS connection (1): s3.amazonaws.com:443
2025-01-23 10:05:48,440 https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/metrics/evaluate-metric/bleu/evaluate-metric/bleu.py HTTP/11" 404 0
2025-01-23 10:05:48,444 Starting new HTTPS connection (1): huggingface.co:443
2025-01-23 10:05:48,761 https://huggingface.co:443 "HEAD /spaces/evaluate-metric/bleu/resolve/v0.4.3/bleu.py HTTP/11" 404 0
2025-01-23 10:05:48,766 Starting new HTTPS connection (1): huggingface.co:443
2025-01-23 10:05:49,254 https://huggingface.co:443 "HEAD /spaces/evaluate-metric/bleu/resolve/main/bleu.py HTTP/11" 200 0
2025-01-23 10:05:49,275 Starting new HTTPS connection (1): github.com:443
2025-01-23 10:05:49,697 https://github.com:443 "HEAD /tensorflow/nmt/raw/master/nmt/scripts/bleu.py HTTP/11" 302 0
2025-01-23 10:05:49,698 Starting new HTTPS connection (1): raw.githubusercontent.com:443
2025-01-23 10:05:50,006 https://raw.githubusercontent.com:443 "HEAD /tensorflow

In [16]:
timestr = time.strftime("%Y%m%d-%H%M%S")
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
training_args = transformers.TrainingArguments(
    output_dir=f"./results/{dataset_name}_{only_model_name}/{timestr}/",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    bf16=torch.cuda.get_device_capability(torch.cuda.current_device())[0] >= 8,  
    gradient_accumulation_steps=4,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    #save_total_limit=2,
    logging_dir='./logs',
    logging_steps=32,
    warmup_steps=100, 
    #eval_steps=32, 
    #max_steps=512,
    
    #load_best_model_at_end=True,
)
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
    data_collator=data_collator,
    #compute_metrics=compute_metrics,
)

In [17]:
trainer.train()
#trainer.evaluate()

Epoch,Training Loss,Validation Loss
1,6.514700,1.617451
2,5.975800,1.504940
3,5.720300,1.455948
4,5.561500,1.427797
5,5.343900,1.412708
6,5.377700,1.404954
7,5.251800,1.402332


2025-01-23 10:38:40,378 Resetting dropped connection: huggingface.co
2025-01-23 10:38:40,824 https://huggingface.co:443 "HEAD /MTSAIR/Cotype-Nano/resolve/main/config.json HTTP/11" 200 0
2025-01-23 10:38:41,010 https://huggingface.co:443 "HEAD /MTSAIR/Cotype-Nano/resolve/main/config.json HTTP/11" 200 0
2025-01-23 11:11:21,867 Resetting dropped connection: huggingface.co
2025-01-23 11:11:22,209 https://huggingface.co:443 "HEAD /MTSAIR/Cotype-Nano/resolve/main/config.json HTTP/11" 200 0
2025-01-23 11:11:22,371 https://huggingface.co:443 "HEAD /MTSAIR/Cotype-Nano/resolve/main/config.json HTTP/11" 200 0
2025-01-23 11:44:02,101 Resetting dropped connection: huggingface.co
2025-01-23 11:44:02,464 https://huggingface.co:443 "HEAD /MTSAIR/Cotype-Nano/resolve/main/config.json HTTP/11" 200 0
2025-01-23 11:44:02,623 https://huggingface.co:443 "HEAD /MTSAIR/Cotype-Nano/resolve/main/config.json HTTP/11" 200 0
2025-01-23 12:27:57,072 Resetting dropped connection: huggingface.co
2025-01-23 12:27:57,50

TrainOutput(global_step=5320, training_loss=5.8286157357065305, metrics={'train_runtime': 14531.459, 'train_samples_per_second': 5.858, 'train_steps_per_second': 0.366, 'total_flos': 1.7373375529746432e+17, 'train_loss': 5.8286157357065305, 'epoch': 7.0})

In [125]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 5.12


In [18]:
final_path = sft_model_path + f"_{timestr}"
model.save_pretrained(final_path)
tokenizer.save_pretrained(final_path)

2025-01-23 14:16:24,142 Resetting dropped connection: huggingface.co
2025-01-23 14:16:24,795 https://huggingface.co:443 "HEAD /MTSAIR/Cotype-Nano/resolve/main/config.json HTTP/11" 200 0
2025-01-23 14:16:24,982 https://huggingface.co:443 "HEAD /MTSAIR/Cotype-Nano/resolve/main/config.json HTTP/11" 200 0


('/app/models/bb_ru_Cotype-Nano_20250123-100550/tokenizer_config.json',
 '/app/models/bb_ru_Cotype-Nano_20250123-100550/special_tokens_map.json',
 '/app/models/bb_ru_Cotype-Nano_20250123-100550/vocab.json',
 '/app/models/bb_ru_Cotype-Nano_20250123-100550/merges.txt',
 '/app/models/bb_ru_Cotype-Nano_20250123-100550/added_tokens.json',
 '/app/models/bb_ru_Cotype-Nano_20250123-100550/tokenizer.json')

In [25]:
final_path

'/app/models/bb_ru_Cotype-Nano_20250123-100550'

In [19]:
reference_model = transformers.AutoModelForCausalLM.from_pretrained(model_name, device_map=device,quantization_config=bnb_config,)

2025-01-23 14:16:45,953 https://huggingface.co:443 "HEAD /MTSAIR/Cotype-Nano/resolve/main/config.json HTTP/11" 200 0
2025-01-23 14:17:10,883 https://huggingface.co:443 "HEAD /MTSAIR/Cotype-Nano/resolve/main/generation_config.json HTTP/11" 200 0


In [20]:
from IPython.display import HTML, display
table_template = """<table style="border:1px solid black" >
  <tr>
    <th style="text-align: center; border:1px solid black">PROMPT</th>
    <th style="text-align: center; border:1px solid black">BEFORE</th>
    <th style="text-align: center; border:1px solid black">AFTER</th>
  </tr>
{}
</table>"""

row_template = '''  <tr>
    <td style="width:20%; border:1px solid black"><pre align="left">`{}`</pre></td>
    <td style="width:40%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:40%; border:1px solid black"><pre align="left">{}</pre></td>
  </tr>'''

def prompt_to_chat(prompt):
    messages = [
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return text

def infer(model, prompt, l=100, use_chat = True, temperature=0.4, top_p = 0.8):
    if use_chat:
        prompt = prompt_to_chat(prompt)
    model_inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=l,
        temperature=temperature, 
        top_p=top_p,
        do_sample=True ,  
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=False)[0]
    return response


prompts =  [
    'Как в BlackBox можно вывести данные на консоль?', 
    'Какой тип данных в Component Pascal используется для хранения целых чисел?', 
    'МОДУЛЬ i21егэDemo2010C4ru;', 
    'Component Pascal is Oberon microsystems refinement of?', 
    'Log.String(', 
    'Типом целой константы является'
]  # feel free to add a few more that are not 100% assiciated with Python

In [21]:
def draw_diff(use_chat,l):
    print(f"USING CHAT TEMPLATE = {use_chat}")
        
    baseline = [infer(reference_model, p, l = l, use_chat=use_chat) for p in prompts]
    check = [infer(model, p, l = l, use_chat=use_chat) for p in prompts]
    rows = []
    for i, prompt in enumerate(prompts):
        # replace placeholders in the format() arguments
        rows.append(row_template.format(prompt, baseline[i], check[i]))
    display(HTML(table_template.format('\n'.join(rows))))

In [22]:
draw_diff(True, 39)

USING CHAT TEMPLATE = True


PROMPT,BEFORE,AFTER
`Как в BlackBox можно вывести данные на консоль?`,"В BlackBox, который является инструментом для тестирования интерфейсов с использованием браузера и эмулятора устройств, вывод данных на кон","Чтобы вывести данные на консоль в BlackBox, используйте процедуры StdLog. Например, чтобы вывести строку ""Hello World"", выполните следующее"
`Какой тип данных в Component Pascal используется для хранения целых чисел?`,"В Pascal, как и во многих других языках программирования, для хранения целых чисел обычно используется тип данных `integer`. Этот тип поддерживает ноль, одно,","В языке программирования Component Pascal целые числа могут быть представлены с использованием различных типов. Например, 32-битные целые числа могут быть представлен"
`МОДУЛЬ i21егэDemo2010C4ru;`,"Ваш запрос ""MODUль i21egэDemo2010C4ru"" кажется неполным или содержит опечатки. Возможно, вы имели в","Этот модуль предназначен для демонстрации работы с модулем i21eduЧерепашка. Он не является обязательным к использованию, но может быть пол"
`Component Pascal is Oberon microsystems refinement of?`,"Oberon-MicroSystems is a company that specializes in the development of microsystem components for industrial applications. It appears there might be some confusion, as ""Pascal"" could refer to different","Oberon Microsystems, Inc. was a small company that specialized in the development of software for microcomputers. It was founded by Wirth and his colleagues at ETH Zurich (Eid"
`Log.String(`,"It seems like you're asking about the `String` method in a logging framework, but your question is incomplete. The `String` method typically refers to converting an object to a string representation in","String(""log""); Log.Ln String(""log"") + Log.Ln String(""log"") + String(""log"") + Log.Ln String(""log"") + String(""log"
`Типом целой константы является`,"Вопрос о ""типом целой константы"" может быть немного неоднозначным, так как термин ""константа"" в контексте программирования обычно","тип INTEGER. Константа может быть объявлена как CONSTANT, например: CONST a = 10; Константам назначаются значения, которые не могут измен"


In [23]:
draw_diff(False, 39)

USING CHAT TEMPLATE = False


PROMPT,BEFORE,AFTER
`Как в BlackBox можно вывести данные на консоль?`,"In the BlackBox framework, how can I output data to the console? To achieve this in BlackBox, you can use the `ConsoleLogger` class which provides a convenient way to log messages","Как это делается? В каких случаях не следует использовать консоль для вывода данных? Команды, которые позволяют работать с данными в консоли, наход"
`Какой тип данных в Component Pascal используется для хранения целых чисел?`,"В компоненте Pascal, для хранения целых чисел обычно используется тип `Int`. Этот тип представляет собой 32-битное беззнаковое целое число. Если",Это 32-битное или 64-битное целое? В языке программирования Компонентный Паскаль целые числа могут быть представлены
`МОДУЛЬ i21егэDemo2010C4ru;`,2010C4ru;2010C4ru;2010C4ru;2010C4ru;2010C4ru,"(* info21, 2010-03-18 *) ПОДКЛЮЧИТЬ Вывод := i21eduВыв"
`Component Pascal is Oberon microsystems refinement of?`,"Oberon is a high-level, multi-paradigm object-oriented programming language. It was developed by the National Research Council of Canada (NRC) and has been used for various applications",1993. The following table gives a brief overview of the differences between Component Pascal and Oberon. The differences are marked with an asterisk (*). Oberon Component Pascal Object
`Log.String(`,1024) .SetMaxSize(stringLen + 5) // Set the maximum size to the sum of the current length and a buffer size .Build(); ```,s ); Log.Ln; END Do; END ObxViews15. Листинг 1-15. Объявление и использование процедуры В этом пример
`Типом целой константы является`,"число 0. Which is correct? The statement ""The type of the whole number 0 is a constant"" is not entirely accurate because while 0 is indeed a whole number, it does","INTEGER. Стандартные типы, которые являются дочерними к INTEGER, это BYTE, SHORTINT, LONGINT и REAL. Другие типы могут быть объ"


In [24]:
print(infer(model, 'Ты помошник по среде Component Pascal. Ответь на вопрос: Как мне вывести строку hello в окно лога?', 256, use_chat = True))

Для вывода строки в окно лога нужно использовать процедуру StdLog.String, которая принимает параметр типа String и помещает его содержимое в окно лога. Следующий пример демонстрирует эту процедуру:

MODULE  i21примЛог;

	IMPORT  Log := StdLog;
	
	PROCEDURE Показать* ( s: ARRAY OF CHAR );
	BEGIN
		Log.String( s ); (* или просто Log.out *)
	END Показать;
	
END  i21примЛог.

Выводится следующее:
	i21примЛог.Показать("hello")

См. также модуль Log для более подробной информации о работе с логом.



In [8]:
checkpoint = transformers.AutoModelForCausalLM.from_pretrained("./results/20250122-220441/checkpoint-1220/", device_map=device)

In [14]:
print(infer(checkpoint, 'Какой тип использовать для хранения целых чисел?', 256, use_chat = True))

Для целых чисел обычно используют тип INTEGER. Однако, если значения могут быть только некоторым предопределенным количеством значений, следует использовать тип SHORTINT, чтобы избежать возможных потерь точности, которые могут возникнуть при делении на 2. В некоторых случаях лучше использовать LONGINT.

Все последующие примеры использует LONGINT в качестве основного типа. Если результаты вычислений становятся непредсказуемыми, если используется другой тип.

Пример:
	
	PROCEDURE Ddouble* (a, b: LONGINT);
	BEGIN
		DIAGNOSTIC;
		StdLog.Nl;
		StdLog.String("Ddouble "); StdLog.Int(a + b, 0); StdLog.Ln;
		StdLog.String(DefaCT(a + b)); StdLog.Ln
	END Ddouble;

В этом примере использовано значение эффекта (defacto) стандартного модуля Math, который включает в себя итоговое значение.
После применения команд DIAGNOSTIC (см. ниже), он будет выровнен по левому краю, а не
